In [13]:
import torch
import torch.nn as nn
import warnings
import torch
import random
warnings.filterwarnings("ignore")

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(777)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        conv1 = nn.Conv2d(1, 3, 3, 1, padding=True) # 1, 6 은 임의로 한것
        conv2 = nn.Conv2d(3, 6, 3, 1, padding=True)
        max_pool = nn.MaxPool2d(2)
        
        fc1 = nn.Linear(3*1*75, 2) # fully connected layer 1번 
        #out = nn.Linear(32, 2) #out put layer 
        
        # Batch x Channel x Height x Width
        self.conv = nn.Sequential(
            conv1,  # N x 1 x 7 x 300 -> N x 3 x 7 x 300
            nn.ReLU(),
            max_pool,  # N x 3 x 7 x 300 -> N x 3 x 3 x 150
            max_pool, # N x 3 x 3 x 150 -> N x 3 x 1 x 75
            #conv2,  # N x 6 x 3 x 150 -> N x 16 x 3 x 150
            #nn.ReLU(),
            #max_pool,  # N x 16 x 3 x 150 -> N x 16 x 1 x 75
        )
        
        self.fc = nn.Sequential(
            fc1,  # N x 1024 -> N x 32
            #nn.ReLU(),# activate function 이걸 뺀것
            #out,  # N x 32 -> N x 2 이걸 뺀것 
            nn.Softmax()  # 이건 분류 할때 사용 한거  out 이 1개일땐 다른거 쓰면 
        )

    def forward(self, x):
        x = self.conv(x)
        
        dim = 1
        for d in x.size()[1:]: #  N x 16 x 1 x 128
            dim = dim * d
            
        x = x.view(-1, dim)
        x = self.fc(x)
        return x

In [17]:
import numpy as np
from EEGDataset import EEGDataset
from torch.utils.data import DataLoader
import torch
import os, glob
from keras.models import load_model
from sklearn.metrics import accuracy_score
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

model = CNN().to(DEVICE)

criterion = nn.BCELoss()
def Accurancy(filename):
    #filename = "./resampled_VR300_0315_Minchul.npz"
    npzfile = np.load(filename)

    Test_data = npzfile['arr_0']
    non_Test_data = npzfile['arr_1']

    #print("Target data shape: ", Test_data.shape)
    #print("Non-target data shape: ", non_Test_data.shape)

    XC_test = np.concatenate([Test_data, non_Test_data])
    YC_test = np.concatenate([np.ones(Test_data.shape[0]), np.zeros(non_Test_data.shape[0])])
    #print("\nX_test shape: ", XC_test.shape)
    #print("y_test shape: ", YC_test.shape)

    batch_size = 4
    epochs=1
    testC_loader = DataLoader(
                    EEGDataset(XC_test, YC_test),batch_size=batch_size, shuffle=False)


    Classifier_path = 'D:\\P300_biosemi_55\\Model\\'
    classifier_list = sorted(glob.glob(Classifier_path + '*.pt'), key=os.path.getmtime, reverse=True)
    weight_list = sorted(glob.glob(Classifier_path + 'weight\\*.pt'), key=os.path.getmtime, reverse=True)
    model = torch.load(classifier_list[0])
    model.load_state_dict(torch.load(weight_list[0]))


    test_loss_list = []
    test_acc_list = []


    for epoch in range(epochs):

        test_loss = 0
        test_acc = 0
        with torch.no_grad():
            for j, Minchul_0315_Test_data in enumerate(testC_loader):
                test_data, test_label = Minchul_0315_Test_data['data'].float().to(DEVICE), Minchul_0315_Test_data['labels'].float().to(DEVICE)
                test_output = model(test_data)
                loss = criterion(test_output, test_label)
                test_loss += loss.item()
                test_acc += accuracy_score(np.argmax(test_label.cpu().detach().numpy(), axis=1), np.argmax(test_output.cpu().detach().numpy(), axis=1))
            
        print('{} test_loss: {:.4f} | acc: {:.4f}'.format(
              filename,test_loss / len(testC_loader), test_acc / len(testC_loader)))
            

        test_loss_list.append(test_loss / len(testC_loader))
        test_acc_list.append(test_acc / len(testC_loader))
        train_loss = 0
        train_acc = 0
        return 1

def main():
    root = './resampled_S'
    for i in np.arange(1,21):
        if(i<10):
            filename = root + '0' + str(i)+'.npz'
        else:
            filename = root + str(i)+'.npz'
        
        Accurancy(filename)
    
if __name__ == "__main__":
    main()

./resampled_S01.npz test_loss: 0.5526 | acc: 0.8567
./resampled_S02.npz test_loss: 0.5498 | acc: 0.8573
./resampled_S03.npz test_loss: 0.5488 | acc: 0.8560
./resampled_S04.npz test_loss: 0.5547 | acc: 0.8573
./resampled_S05.npz test_loss: 0.5597 | acc: 0.8560
./resampled_S06.npz test_loss: 0.5571 | acc: 0.8573
./resampled_S07.npz test_loss: 0.5566 | acc: 0.8573
./resampled_S08.npz test_loss: 0.5515 | acc: 0.8573
./resampled_S09.npz test_loss: 0.5554 | acc: 0.8573
./resampled_S10.npz test_loss: 0.5577 | acc: 0.8560
./resampled_S11.npz test_loss: 0.5513 | acc: 0.8573
./resampled_S12.npz test_loss: 0.5492 | acc: 0.8580
./resampled_S13.npz test_loss: 0.5585 | acc: 0.8573
./resampled_S14.npz test_loss: 0.5572 | acc: 0.8580
./resampled_S15.npz test_loss: 0.5638 | acc: 0.8567
./resampled_S16.npz test_loss: 0.5518 | acc: 0.8573
./resampled_S17.npz test_loss: 0.5514 | acc: 0.8573
./resampled_S18.npz test_loss: 0.5577 | acc: 0.8573
./resampled_S19.npz test_loss: 0.5570 | acc: 0.8573
./resampled_